In [ ]:
# Analyzing Impact of Knowledge Base Bias in RAG on LLM Fake News Classification

### **Configuration**

In [ ]:
# Dependencies
import os
from config import DARTMOUTH_API_KEY, DARTMOUTH_CHAT_API_KEY
from langchain_dartmouth.llms import ChatDartmouth

# Retrieving keys and creating environment variables
os.environ['DARTMOUTH_CHAT_API_KEY'] = DARTMOUTH_CHAT_API_KEY
os.environ['DARTMOUTH_API_KEY'] = DARTMOUTH_API_KEY

# Defining chat model
model_name = "openai.gpt-4o-mini-2024-07-18"

# Defining embeddings model
# Defining vector store

### **Building Knowledge Base**

In [ ]:
# Defining keywords and sources
keywords = "Trump"
source="cnn"

In [17]:
# Importing dependencies
import requests
import os
from bs4 import BeautifulSoup
from newsapi import NewsApiClient
from config import NEWS_API_KEY

# Creating directory to hold scraped articles
os.makedirs(name=f"{keywords}_{source}")

# Creating newsapi client
newsapiclient = NewsApiClient(api_key=NEWS_API_KEY)

# Querying articles
articles = newsapiclient.get_everything(q=keywords, sources=source ,page_size=1)
articles = articles['articles']

# Scraping articles and saving in directory
for article in articles:
	url = article['url']
	response = requests.get(url=url)

	# Printing article content to directory if valid response
	if response.status_code==200:
		beautifulsoup = BeautifulSoup(response.content, "html.parser")
		article_paragraphs = beautifulsoup.find_all("p")

		# Cleaning article title

		with open(file=f"{keywords}_{source}/{article['title']}.txt", mode="w") as fp:
			for paragraph in article_paragraphs:
				fp.write(str(paragraph.get_text()))

### **Loading Documents**

In [ ]:
# import nltk
# nltk.download('averaged_perceptron_tagger')

In [ ]:
# Importing dependencies
from langchain.document_loaders import DirectoryLoader

# Defining directory path
directory = f"{keywords}_{source}"

# Loading documents
loader = DirectoryLoader(path=directory, glob="*.txt")
docs = loader.load()

for doc in docs:
    print(doc)




libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


page_content='Follow: The Trump administration has its government-shrinking sights set on the National Oceanic and Atmospheric Administration, where around 800 employees have been tapped for termination, according to two sources close to the agency.

More layoffs are possible Friday, one of the sources said, potentially costing the weather, climate and environmental agency more than a thousand employees by the end of the week.

Most divisions of the agency, which employs scientists and specialists in weather, oceans, biodiversity, climate and other research and planetary monitoring fields, were affected.

Probationary employees — those who have been in their jobs for a year or less, in most cases — were fired Thursday, a person inside the National Weather Service told CNN. There are between 350 and 375 employees with that status at the weather service, though it’s not clear how many of those were impacted; the person has heard there were some exemptions given to critical positions — li

### **Splitting Documents**

### **Storing Documents**

### **Retrieval**

### **Generation**

### **Evaluation**

In [ ]:
from langchain_dartmouth.llms import ChatDartmouthCloud

llm = ChatDartmouthCloud(model_name="openai.gpt-4o-mini-2024-07-18")

# response = llm.invoke("Who are you?")

# print(response.content)

# User's question
query = '''Lina Khan has without doubt been the worst FTC chair in history'''

# Context relevant to the question from our knowledge base
relevant_document = ''' 
From WSJ: Ms. Khan, the chairwoman of the Federal Trade Commission, has thrown way too much water on legitimate businesses and commerce in this country. We’ll never know how many times boards of directors have turned down good ideas because they don’t want to waste money in court fighting a stupid ruling from Ms. Khan’s FTC.

'''

# Augment prompt
augmented_prompt = (
	relevant_document + "Considering this from a news source, answer the following question:" + query
)

# Generate the answer
response = llm.invoke(augmented_prompt)
response.pretty_print()

In [ ]:
from langchain_dartmouth.llms import ChatDartmouth

llm = ChatDartmouth(model_name=model_name)

# User's question
query = '''Fake or Real News: Trump is bad for the climate'''

# Context relevant to the question from our knowledge base
relevant_document = ''' 
From WSJ: Ms. Khan, the chairwoman of the Federal Trade Commission, has thrown way too much water on legitimate businesses and commerce in this country. We’ll never know how many times boards of directors have turned down good ideas because they don’t want to waste money in court fighting a stupid ruling from Ms. Khan’s FTC.

'''

# Augment prompt
augmented_prompt = (
	relevant_document + "Considering this from a news source, answer the following question:" + query
)

# Generate the answer
response = llm.invoke(augmented_prompt)
response.pretty_print()